# Election Statistics ANALYSIS

## Importing Libraries

In [1]:
from pandas import DataFrame,read_csv,concat,options,set_option,to_datetime,to_numeric
from os import listdir

In [2]:
options.display.float_format = "{:,.2f}".format
set_option('display.max_rows',None)

## National/State Summaries

### Load Data

In [3]:
csv_files:list[str] = listdir('State_Summary_Results')
data:dict[str,DataFrame] = {}
df:DataFrame = DataFrame(['State','KH_Vote_Count', 'KH_Vote_Pct', 'DT_Vote_Count', 'DT_Vote_Pct','Pct_Reported'])

In [4]:
for file in csv_files:
    if('FINAL' in file):
        data[file.replace('_Results.csv','').replace('FINAL_','')] = read_csv(f"State_Summary_Results/{file}")

### Merge state summaries into one giant national table

In [5]:
for key,item in data.items():
    item.insert(loc=1,column='State',value=key)
    df:DataFrame = concat([df,item],axis=0)

In [6]:
all_states_summary_df = df.drop([0],axis=1).reset_index(drop=True).iloc[6:]

### Add Nationwide Total Vote Count Columns

In [7]:
total_votes:DataFrame = all_states_summary_df.groupby('Datetime_Extraction').sum(numeric_only=True)

In [8]:
total_votes['Nationwide_Total_Votes'] = total_votes['KH_Vote_Count']+total_votes['DT_Vote_Count']
total_votes['Additional_Votes_Since_Last_Update'] = total_votes['Nationwide_Total_Votes'].diff()
#total_votes['Average_Additional_Votes_Per_Minute'] = total_votes['Additional_Votes_Since_Last_Update'].fillna(0).astype(float)/to_datetime(total_votes.index.to_series()).diff()

In [9]:
vote_updates:DataFrame = total_votes.reset_index(drop=False).groupby('Nationwide_Total_Votes').first().reset_index()[['Datetime_Extraction','Nationwide_Total_Votes','KH_Vote_Count','DT_Vote_Count']]

In [10]:
vote_updates['Vote_Change'] = vote_updates['Nationwide_Total_Votes'].diff().fillna(0)

In [11]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].to_csv('ANALYSIS/Nationwide_Vote_Count_Updates.csv',index=True)

In [12]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2443 entries, 2024-11-05 17:06:07.133829 to 2024-11-11 14:20:55.707590
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   KH_Vote_Count                       2443 non-null   float64
 1   DT_Vote_Count                       2443 non-null   float64
 2   Nationwide_Total_Votes              2443 non-null   float64
 3   Additional_Votes_Since_Last_Update  2443 non-null   float64
dtypes: float64(4)
memory usage: 95.4+ KB


### Group State Summaries By Vote Count Changes

In [15]:
trump_total_negative_votes = 0
harris_total_negative_votes = 0
for key,item in data.items():
    temp_df:DataFrame = item.copy()
    temp_df.insert(5,'Total_Votes',temp_df['KH_Vote_Count']+temp_df['DT_Vote_Count'])
    temp_df:DataFrame = temp_df.groupby('Total_Votes').first().reset_index(drop=False)
    temp_df:DataFrame = temp_df.sort_values(by=['Datetime_Extraction'],axis=0).reset_index(drop=True)
    temp_df['Additional_Votes_Since_Last_Update'] = temp_df['Total_Votes'].diff()
    temp_df['Additional_Trump_Votes_Since_Last_Update'] = temp_df['DT_Vote_Count'].diff()
    temp_df['Additional_Harris_Votes_Since_Last_Update'] = temp_df['KH_Vote_Count'].diff()
    temp_df['Pct_New_Votes_For_Trump'] = (temp_df['Additional_Trump_Votes_Since_Last_Update']/temp_df['Additional_Votes_Since_Last_Update'])*100
    temp_df['Pct_New_Votes_For_Harris'] = (temp_df['Additional_Harris_Votes_Since_Last_Update']/temp_df['Additional_Votes_Since_Last_Update'])*100
    temp_df:DataFrame = temp_df.fillna(0)
    cols = list(temp_df)
    cols.insert(0, cols.pop(cols.index('Datetime_Extraction')))
    temp_df = temp_df.loc[:, cols]
    for col in temp_df.columns:
        try:
            temp_df[col] = temp_df[col].astype(float)
        except:pass
    temp_df.to_csv(f'ANALYSIS/State_Vote_Count_Updates/{key}_Vote_Count_Updates.csv',index=True,float_format='%.2f')
    negative_vote_changes:DataFrame = temp_df[(temp_df['Additional_Harris_Votes_Since_Last_Update']<0)|
                                              (temp_df['Additional_Trump_Votes_Since_Last_Update']<0)]
    if(len(negative_vote_changes)>0):
        harris_total_negative_votes += negative_vote_changes[(negative_vote_changes['Additional_Harris_Votes_Since_Last_Update']<0)]['Additional_Harris_Votes_Since_Last_Update'].sum()
        trump_total_negative_votes += negative_vote_changes[(negative_vote_changes['Additional_Trump_Votes_Since_Last_Update']<0)]['Additional_Trump_Votes_Since_Last_Update'].sum()
        print(f"{key}\n"\
            f"Harris Negative Total: {negative_vote_changes[(negative_vote_changes['Additional_Harris_Votes_Since_Last_Update']<0)]['Additional_Harris_Votes_Since_Last_Update'].sum():,.0f}\n"\
            f"Trump Negative Total: {negative_vote_changes[(negative_vote_changes['Additional_Trump_Votes_Since_Last_Update']<0)]['Additional_Trump_Votes_Since_Last_Update'].sum():,.0f}\n")
    negative_indexes:list = list(negative_vote_changes.index.copy())
    final_indexes:list[int] = []
    for index in negative_indexes:
        final_indexes.append(index-1)
        final_indexes.append(index)
        final_indexes.append(index+1)
    if(len(negative_vote_changes.index)>0):
        try:
            temp_df.iloc[final_indexes].drop_duplicates().to_csv(f"ANALYSIS/Negative_Votes/{key}_NEGATIVE_Vote_Count_Updates.csv",index=True,float_format="%.2f")
        except:
            temp_df.iloc[final_indexes[:-1]].drop_duplicates().to_csv(f"ANALYSIS/Negative_Votes/{key}_NEGATIVE_Vote_Count_Updates.csv",index=True,float_format="%.2f")
        final_indexes.clear()
        final_indexes:list[int] = []
    final_indexes.clear()
    final_indexes:list[int] = []

Arkansas
Harris Negative Total: -95
Trump Negative Total: -8,205

Connecticut
Harris Negative Total: -5,977
Trump Negative Total: -3,844

Florida
Harris Negative Total: -3,858
Trump Negative Total: -2,184

Georgia
Harris Negative Total: -1,049
Trump Negative Total: -17

Idaho
Harris Negative Total: -2,299
Trump Negative Total: 0

Indiana
Harris Negative Total: -11,229
Trump Negative Total: -616

Iowa
Harris Negative Total: -1
Trump Negative Total: 0

Kansas
Harris Negative Total: -223
Trump Negative Total: -207

Kentucky
Harris Negative Total: -10,571
Trump Negative Total: 0

Maryland
Harris Negative Total: -64
Trump Negative Total: -14

Massachusetts
Harris Negative Total: -1,566
Trump Negative Total: -3,621

Michigan
Harris Negative Total: -91,717
Trump Negative Total: -166,812

Minnesota
Harris Negative Total: -14,959
Trump Negative Total: -10,394

Mississippi
Harris Negative Total: 0
Trump Negative Total: -2,874

Montana
Harris Negative Total: -364
Trump Negative Total: -2,552

Nev

In [14]:
print(f"Harris Negative Votes Total: {harris_total_negative_votes:,}\nTrump Negative Votes Total: {trump_total_negative_votes:,}")

Harris Negative Votes Total: -389,992.0
Trump Negative Votes Total: -425,281.0
